In [ ]:
using ITensors, ITensorMPS
import Pkg
Pkg.activate(@__DIR__)
let
    # Create 100 spin-one indices
    N = 100
    sites = siteinds("S=1", N)

    # Input operator terms which define
    # a Hamiltonian matrix, and convert
    # these terms to an MPO tensor network
    # (here we make the 1D Heisenberg model)
    os = OpSum()
    for j in 1:(N - 1)
        os += "Sz", j, "Sz", j + 1
        os += 0.5, "S+", j, "S-", j + 1
        os += 0.5, "S-", j, "S+", j + 1
    end
    H = MPO(os, sites)

    # Create an initial random matrix product state
    psi0 = random_mps(sites)

    # Plan to do 5 passes or 'sweeps' of DMRG,
    # setting maximum MPS internal dimensions
    # for each sweep and maximum truncation cutoff
    # used when adapting internal dimensions:
    nsweeps = 5
    maxdim = [10, 20, 100, 100, 200]
    cutoff = 1.0e-10

    # Run the DMRG algorithm, returning energy
    # (dominant eigenvalue) and optimized MPS
    energy, psi = dmrg(H, psi0; nsweeps, maxdim, cutoff)
    println("Final energy = $energy")

    nothing
end

# output

# After sweep 1 energy=-137.954199761732 maxlinkdim=9 maxerr=2.43E-16 time=9.356
# After sweep 2 energy=-138.935058943878 maxlinkdim=20 maxerr=4.97E-06 time=0.671
# After sweep 3 energy=-138.940080155429 maxlinkdim=92 maxerr=1.00E-10 time=4.522
# After sweep 4 energy=-138.940086009318 maxlinkdim=100 maxerr=1.05E-10 time=11.644
# After sweep 5 energy=-138.940086058840 maxlinkdim=96 maxerr=1.00E-10 time=12.771
# Final energy = -138.94008605883985

  Activating new project at `c:\Users\Aradh\Desktop\coding\697_fall_2025_Max_Aradh_project`


After sweep 1 energy=-0.3191767339069923  maxlinkdim=8 maxerr=7.21E-11 time=0.099
After sweep 2 energy=-0.16104432134520164  maxlinkdim=20 maxerr=1.29E-04 time=0.244
After sweep 3 energy=-6.953420041631511e-6  maxlinkdim=100 maxerr=5.67E-09 time=2.536
After sweep 4 energy=-0.22322805023552483  maxlinkdim=100 maxerr=1.37E-05 time=6.838
After sweep 5 energy=-0.3286949963582739  maxlinkdim=200 maxerr=8.12E-07 time=20.941
Final energy = -0.3286949963582739
